data= pd.read_csv("") charger le dataset 

In [3]:
# recuper 10% du dataset pour alléger la mémoire  
data_alleger=data.sample(frac=0.1) 


  fonction qui donne des informations pour aider à décider quelle feature on doit conserver, dans le cas où
on a 2 features qui semblent correspondre à la même notion.

In [ ]:
def compare_na(df, col1, col2):    
#Elle remonte 3 informations :
# de cas où la valeur de la colonne 1 est renseignée, mais pas la 2
#de cas où la valeur de la colonne 2 est renseignée, mais pas la 1
#de cas où les valeurs de la colonne 1 et 2 sont renseignées toutes les deux

    num_rows, num_cols = df.shape
    
    col1notnull_col2null = round ( ( ( ( df[ (df[col1].notnull()) & ( df[col2].isna() ) ][[col1,col2]].shape[0] ) / num_rows ) * 100), 5)
    col2notnull_col1null = round ( ( ( ( df[ (df[col2].notnull()) & ( df[col1].isna() ) ][[col1,col2]].shape[0] ) / num_rows ) * 100), 5)
    col1notnull_col2notnull = round ( ( ( ( df[ (df[col1].notnull()) & ( df[col2].notnull() ) ][[col1,col2]].shape[0] ) / num_rows ) * 100), 5)
    
    print(f'Cas où {col1} est renseigné mais pas {col2} : {col1notnull_col2null}%')
    print(f'Cas où {col2} est renseigné mais pas {col1} : {col2notnull_col1null}%')
    print(f'Cas où {col1} et {col2} sont renseignés tous les deux : {col1notnull_col2notnull}%')

Analyse plus détaillée des données et des colonnes manquantes

In [ ]:
def analyse_donnees_manquantes(df, seuil = .7):
    nb_rows, nb_cols = df.shape

    nb_col_many_nulls = (((df.isnull().sum()) / nb_rows) > seuil)

    percentage_col_many_nulls = round(((nb_col_many_nulls.sum()) / nb_cols) * 100, 2)

    message = ( 
        f"{percentage_col_many_nulls} % des colonnes ont >= {seuil*100:0.0f}% de données manquantes \n"  
        f"Ces colonnes sont : \n"
        f"{nb_col_many_nulls[nb_col_many_nulls].to_string()}"
    )

    print(message)
    
analyse_donnees_manquantes(food)

suppression des features dont le nombre de données est inférieur au seuil minimum défini

In [ ]:
min_percentage_feature_values_tokeep = collections.defaultdict(lambda : 0.01)
#min_percentage_feature_values_tokeep['vitamin-b2_100g'] = 0.01  # Use this to have specific percentage value 

def drop_lowquality_values(df, min_percentage_feature_values_tokeep):
    num_rows, num_cols = df.shape
    
    for column_name in df.columns:
        if (len(food[food[column_name].notnull()]) < min_percentage_feature_values_tokeep[column_name] * num_rows):
            df.drop([column_name], axis='columns', inplace = True)

Nettoyage des données pour les features cibles¶
Conserver uniquement les features retenues¶


In [ ]:
features_list = ['code', 'last_modified_t', 'product_name' , 'states_tags', 'main_category_fr','brands','brands_tags', 'nutriscore_grade','energy_100g','sugars_100g','salt_100g','saturated-fat_100g','fiber_100g','proteins_100g','ingredients_from_palm_oil_n','pnns_groups_2','pnns_groups_1','labels_tags','countries_tags','additives_tags','additives_n','ingredients_text','image_url', 'nova_group']
food = food[features_list]

In [ ]:
#Ajout d'une feature numérique de scoring pour le nutrition grade¶
def convert_category_to_number(cat):
    cat_table = {
        'a' : 5,
        'b' : 4,
        'c' : 3,
        'd' : 2,
        'e' : 1,
        'nan' : None,
    }
    
    return (cat_table.get(cat,None))


food_cat = pd.DataFrame(food['nutriscore_grade'].apply(convert_category_to_number))

food['nutrition_scoring'] = food_cat